In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# نمایش بهتر داده‌ها
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set(style="whitegrid")


In [21]:
# بارگذاری داده
file_path = '../data/Taxi_Trips_2024_clean.csv'
df = pd.read_csv(file_path)

# بررسی اولیه
print("شکل دیتافریم:", df.shape)
print("ستون‌ها:", df.columns.tolist())

# تبدیل timestampها به datetime
df['Trip Start Timestamp'] = pd.to_datetime(df['Trip Start Timestamp'])
df['Trip End Timestamp'] = pd.to_datetime(df['Trip End Timestamp'])

# حذف مقادیر غیرمنطقی یا گمشده در ستون‌های کلیدی
df = df.dropna(subset=['Trip Seconds', 'Trip Miles', 'Fare', 'Trip Total'])

# حذف مقادیر صفر یا منفی که به‌احتمال زیاد خطا هستند
df = df[(df['Trip Seconds'] > 0) & (df['Trip Miles'] > 0) & (df['Fare'] > 0)]

# ریست ایندکس بعد از فیلتر کردن
df.reset_index(drop=True, inplace=True)

# نمایش آماری خلاصه
df.describe()


شکل دیتافریم: (791784, 13)
ستون‌ها: ['Trip ID', 'Taxi ID', 'Trip Start Timestamp', 'Trip End Timestamp', 'Trip Seconds', 'Trip Miles', 'Fare', 'Tips', 'Tolls', 'Extras', 'Trip Total', 'Payment Type', 'Company']


,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Fare,Tips,Tolls,Extras,Trip Total
count,711775,711775,711775.00,711775.00,711775.00,711775.00,711775.00,711775.00,711775.00
mean,2024-08-18 10:52:48.302202624,2024-08-18 11:15:00.443820288,1331.60,7.53,23.03,2.96,0.03,2.14,28.37
min,2024-01-01 00:00:00,2024-01-01 00:15:00,1.00,0.01,0.01,0.00,0.00,0.00,0.01
25%,2024-05-03 17:15:00,2024-05-03 17:45:00,540.00,1.41,8.75,0.00,0.00,0.00,10.75
50%,2024-08-16 18:30:00,2024-08-16 19:00:00,1020.00,4.66,17.25,1.00,0.00,0.00,20.04
75%,2024-12-01 17:45:00,2024-12-01 18:15:00,1786.00,12.70,35.00,4.22,0.00,4.00,44.00
max,2025-04-01 00:00:00,2025-04-01 17:00:00,83040.00,2820.67,3668.50,150.00,4444.44,4890.00,8897.13
std,NaN,NaN,1642.06,8.34,18.45,4.21,5.60,9.73,26.80


In [27]:
# انتخاب ویژگی‌ها (Features) و خروجی (Label)
feature_cols = ['Trip Seconds', 'Trip Miles', 'Fare', 'Tips', 'Tolls', 'Extras']
X = df[['Trip Seconds', 'Trip Miles', 'Fare']]
y = df['Trip Total']

# بررسی اولیه برای اطمینان
print("شکل داده‌ها:")
print("X:", X.shape)
print("y:", y.shape)

# تقسیم داده‌ها به آموزش و آزمون
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


شکل داده‌ها:
X: (711775, 3)
y: (711775,)


In [29]:
# تعریف مدل
model = LinearRegression()

# آموزش مدل روی داده‌های آموزشی
model.fit(X_train, y_train)

# بررسی ضرایب مدل
coefficients = pd.DataFrame({
    'ویژگی': X.columns,
    'ضریب': model.coef_
})
print("ضرایب مدل:")
print(coefficients)

# نمایش بایاس (intercept)
print("\nبایاس (intercept):", model.intercept_)


ضرایب مدل:
          ویژگی  ضریب
0  Trip Seconds -0.00
1    Trip Miles  0.19
2          Fare  1.15

بایاس (intercept): 0.48177259363572844


In [31]:
# پیش‌بینی روی داده‌های آزمون
y_pred = model.predict(X_test)

# محاسبه معیارهای ارزیابی
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print("ارزیابی مدل روی داده‌های آزمون:")
print(f"MAE (میانگین خطای مطلق): {mae:.2f}")
print(f"RMSE (ریشه میانگین مربع خطاها): {rmse:.2f}")
print(f"MSE (میانگین مربع خطاها): {mse:.2f}")


ارزیابی مدل روی داده‌های آزمون:
MAE (میانگین خطای مطلق): 4.23
RMSE (ریشه میانگین مربع خطاها): 15.41
MSE (میانگین مربع خطاها): 237.59


In [33]:
# لیست ستون‌هایی که می‌خوایم Outlierهاشون رو حذف کنیم
columns_to_filter = ['Trip Seconds', 'Trip Miles', 'Fare', 'Tips', 'Tolls', 'Extras']

df_filtered = df.copy()

for col in columns_to_filter:
    Q1 = df_filtered[col].quantile(0.25)
    Q3 = df_filtered[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # فیلتر فقط داده‌های داخل بازه مجاز
    df_filtered = df_filtered[(df_filtered[col] >= lower_bound) & (df_filtered[col] <= upper_bound)]

# بررسی بعد از فیلتر
print(f"تعداد داده‌ها بعد از حذف outlierها: {df_filtered.shape[0]}")


تعداد داده‌ها بعد از حذف outlierها: 507454


In [35]:
# آماده‌سازی داده‌ها
X_filtered = df_filtered[['Trip Seconds', 'Trip Miles', 'Fare']]
y_filtered = df_filtered['Trip Total']

# تقسیم به آموزش و تست
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_filtered, y_filtered, test_size=0.2, random_state=42
)

# آموزش مدل
model_f = LinearRegression()
model_f.fit(X_train_f, y_train_f)

# پیش‌بینی روی داده تست
y_pred_f = model_f.predict(X_test_f)

# ارزیابی
mae_f = mean_absolute_error(y_test_f, y_pred_f)
rmse_f = np.sqrt(mean_squared_error(y_test_f, y_pred_f))
mse_f = mean_squared_error(y_test_f, y_pred_f)

print(f"MAE جدید: {mae_f:.2f}")
print(f"RMSE جدید: {rmse_f:.2f}")
print(f"MSE جدید: {mse_f:.2f}")


MAE جدید: 1.79
RMSE جدید: 2.23
MSE جدید: 4.99
